In [10]:
import requests
import pandas as pd
from datetime import timedelta
from datetime import datetime

from airflow import DAG
from airflow.operators.python import PythonOperator


In [11]:

TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'


In [12]:
def get_data():
    # Здесь пока оставили запись в файл, как передавать переменную между тасками будет в третьем уроке
    top_doms = pd.read_csv(TOP_1M_DOMAINS)
    top_data = top_doms.to_csv(index=False)

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)



In [13]:
def get_top10domains():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    domains = top_data_df['domain'].str.split('.').str[-1]
    counts = domains.value_counts()
    top10domains = counts.head(10)
    with open('top10domains.csv', 'w') as f:
        f.write(top10domains.to_csv(index=False, header=False))


In [14]:
def longest():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    longest = sorted(top_data_df[top_data_df['domain'].str.len() == top_data_df['domain'].str.len().max()]['domain'])[0]
    with open('longest.csv', 'w') as f:
        f.write(longest.to_csv(index=False, header=False))


In [15]:
def airflow_pos():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    row = top_doms.loc[top_data_df['domain'] == 'airflow.com']
    airflow_pos = row['rank'].values[0]
    with open('airflow_pos.csv', 'w') as f:
        f.write(airflow_pos.to_csv(index=False, header=False))

In [16]:
def print_data(ds):
    with open('top10domains.csv', 'r') as f:
        top10domains_d = f.read()
    with open('longest.csv', 'r') as f:
        longest_domain = f.read()
    with open('airflow_pos.csv', 'r') as f:
        airflow_pos_s = f.read()
    date = ds

    print(f'Top domains {date}')
    print(top10domains_d)

    print(f'Longest domain {date}')
    print(longest_domain)
    
    
    print(f'Airflow position is {date}')
    print(airflow_pos_s)


In [17]:

default_args = {
    'owner': 'ar-davydov',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 4, 9),
}
schedule_interval = '0 11 * * *'

dag = DAG('some_tasks_kk', default_args=default_args, schedule_interval=schedule_interval)

t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)

t2_10d = PythonOperator(task_id='get_top10domains',
                    python_callable=get_top10domains,
                    dag=dag)

t2_Longest = PythonOperator(task_id='longest',
                        python_callable=longest,
                        dag=dag)

t2_airflow = PythonOperator(task_id='airflow_pos',
                        python_callable=airflow_pos,
                        dag=dag)

t3 = PythonOperator(task_id='print_data',
                    python_callable=print_data,
                    dag=dag)

t1 >> [t2_10d, t2_Longest, t2_airflow] >> t3


<Task(PythonOperator): print_data>